In [7]:
FUENTES_DIR = '../Fuentes'         # carpeta donde se encuentran archivos .py auxiliares
DATOS_DIR   = '../Data_Sets/p2/' # carpeta donde se encuentran los datasets

# agrega ruta de busqueda donde tenemos archivos .py
import sys
sys.path.append(FUENTES_DIR)

%matplotlib inline
import numpy as np
import pandas as pd
from matplotlib import pylab as plt
from IPython import display
from sklearn import preprocessing
import grafica as gr
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from ClassRNMulticlase import RNMulticlase

# Regresión Logística multiclase
## Ejercicio 11

Se busca predecir si el tipo de fármaco que se debe administrar a un paciente afectado de rinitis alérgica es el habitual o no. Se dispone de información correspondiente a las historias clínicas de pacientes atendidos previamente


### a) 

Numerice los atributos nominales y utilice el archivo **drug_train.csv** para entrenar un modelo con 5 neuronas que sea capaz de predecir el tipo de fármaco que se debe administrar a un paciente afectado de rinitis alérgica.

In [36]:
ALPHA = 0.01
N_ITER = 300
COTA_E = 10E-07
FUN = 'sigmoid'
COSTO = 'ECM'

data_train = pd.read_csv(DATOS_DIR + 'drugs_train.csv', sep=',')

# Separo X (features) e y (target)
X = data_train.drop(columns=['Drug'])
T = data_train['Drug'].map({'drugA':0,'drugB':1,'drugC':2,'drugX':3,'drugY':4}).to_numpy()
T_onehot = np.zeros((len(T), 5))
T_onehot[np.arange(len(T)), T] = 1
# Defino columnas categóricas y numéricas
cat_cols = ["Sex", "BP", "Cholesterol"]
num_cols = ["Age", "Na", "K"]

# Preprocesamiento: OneHot para categóricas + passthrough numéricas
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), cat_cols),
        ('num', 'passthrough', num_cols)
    ]
)

# Aplico transformación solo sobre X
X_transformed = preprocessor.fit_transform(X)
X_train = pd.DataFrame(X_transformed.toarray() if hasattr(X_transformed, "toarray") else X_transformed)

# Normalizo por z-score
scaler = preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)

#Entreno la red neuronal
nn = RNMulticlase(alpha=ALPHA, n_iter=N_ITER, cotaE=COTA_E, FUN=FUN,COSTO=COSTO, random_state=None)
nn.fit(X_train, T_onehot)

#Pruebo la red 
y_pred = nn.predict(X_train)
print(y_pred)
accuracy = np.mean(y_pred == T) * 100
print(f'Accuracy en train: {accuracy:.2f}%')

[2 4 4 2 4 0 4 4 4 0 4 0 4 4 4 4 4 4 4 0 3 3 3 4 4 4 4 4 4 3 4 2 4 4 3 0 1
 4 4 3 3 0 4 0 0 1 4 3 0 2 4 1 3 3 3 3 3 4 1 3 3 3 0 3 3 3 3 0 3 4 0 4 4 1
 0 3 4 1 2 3 4 2 4 3 3 0 1 3 4 2 3 0 4 2 4 1 1 4 0 4 0 4 0 2 2 4 0 3 4 3 3
 3 0 0 4 2 4 0 1 3 4 0 4 0 0 4 3 2 4 3 4 4 0 4 3 4 0 4 4 3 1 2 4 3 3 4 4 4
 4 3 3 4 0 4 3 4 4 4 4 4]
Accuracy en train: 90.62%


### b) 

Luego utilice el archivo drug_test.csv para medir la calidad del modelo.

In [37]:
X_test_raw = data_test.drop(columns=['Drug'])

# Usás el preprocessor ya fiteado en train
X_test = preprocessor.transform(X_test_raw)
X_test = pd.DataFrame(X_test.toarray() if hasattr(X_test, "toarray") else X_test)

# Usás el scaler ya fiteado en train
X_test = scaler.transform(X_test)

# Predicción con la red ya entrenada
y_pred_test = nn.predict(X_test)

accuracy_test = np.mean(y_pred_test == T_test) * 100
print(f'Accuracy en test: {accuracy_test:.2f}%')


Accuracy en test: 80.00%
